In [0]:
# ------------------------
# Imports
# ------------------------
import requests as r
import json
from datetime import datetime, UTC


# ------------------------
# Widgets for parameters
# ------------------------

dbutils.widgets.text("city", "bangalore")
city = dbutils.widgets.get("city")

dbutils.widgets.text("token", "")
token = dbutils.widgets.get("token")

# ------------------------
# API Call
# ------------------------
url = "https://api.waqi.info/feed"

try:
    response = r.get(f"{url}/{city}/?token={token}")

except Exception as e:
    raise Exception(f"API call failed due to network error: {str(e)}")

if response.status_code != 200:
    raise Exception(
        f"API returned non-200 status.\n"
        f"Status: {response.status_code}\n"
        f"Message: {response.text}"
    )

try:
    data = response.json()
except Exception:
    raise Exception(f"API returned invalid JSON: {response.text}")

# print(json.dumps(data, indent=4))

# ------------------------
# Add ingestion timestamp
# ------------------------

ingested_at = datetime.now(UTC).isoformat()
data["ingested_at"] = ingested_at

# ------------------------
# Build Bronze Path
# Folder structure: year/month/day/hour/
# ------------------------
now = datetime.now(UTC)
year = now.year
month = str(now.month).zfill(2)
day = str(now.day).zfill(2)
hour = str(now.hour).zfill(2)

# Unique filename using epoch timestamp
epoch_ts = int(now.timestamp())
file_name = f"weather_{epoch_ts}.json"
timestamp = data["data"]["time"]["s"]

bronze_path = f"/Volumes/aqi_etl_pipeline/bronze_aqi/bronze_vol_aqi/aqi/{year}/{month}/{day}/{hour}/{file_name}"


# ------------------------
# Write JSON to Bronze Volume
# ------------------------
try:
    json_str = json.dumps(data)
    dbutils.fs.put(bronze_path, json_str, overwrite=False)
except Exception as e:
    raise Exception(f"Failed to write file to Bronze Volume: {str(e)}")

# ------------------------
# Summary
# ------------------------
print("✔ AQI API ingestion completed")
print(f"✔ File saved at: {bronze_path}")
print(f"✔ Weather timestamp (dt): {timestamp}")
print(f"✔ Ingested at (UTC): {ingested_at}")